In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.spatial as scp

In [ ]:
x = np.linspace(-1, 1, 4)
y = np.linspace(-1, 1, 3)
x_grid, y_grid = np.meshgrid(x, y)
z_grid = np.ones_like(x_grid)

vertices = np.array((x_grid.ravel(), y_grid.ravel())).T
print(vertices.shape)

In [ ]:
tri = scp.Delaunay(vertices)
fig, ax = plt.subplots(1, 1)
ax.axis("equal")
ax.triplot(vertices[:, 0], vertices[:, 1], tri.simplices)

In [ ]:
# Build grid indexes to select respective triangle vertices
def _vertex_indices(xi, yi, xstride):
    return xi + xstride * yi


def triangulate_grid(x, y, flip=False, divide="nesw"):
    """
    Create a 2D triangulation for a regular (x, y) grid.

    Parameters
    ----------
    """
    x_grid, y_grid = np.meshgrid(x, y)
    vertices = np.array((x_grid.ravel(), y_grid.ravel())).T

    xi = np.arange(0, len(x), 1, dtype="int")
    yi = np.arange(0, len(y), 1, dtype="int")

    xg, yg = np.meshgrid(xi[:-1], yi[:-1])
    vertices_sw = _vertex_indices(xg.ravel(), yg.ravel(), xstride=len(x))

    xg, yg = np.meshgrid(xi[1:], yi[:-1])
    vertices_se = _vertex_indices(xg.ravel(), yg.ravel(), xstride=len(x))

    xg, yg = np.meshgrid(xi[:-1], yi[1:])
    vertices_nw = _vertex_indices(xg.ravel(), yg.ravel(), xstride=len(x))

    xg, yg = np.meshgrid(xi[1:], yi[1:])
    vertices_ne = _vertex_indices(xg.ravel(), yg.ravel(), xstride=len(x))

    if divide == "nesw":
        face_indices_1 = np.array((vertices_sw, vertices_ne, vertices_se)).T
        face_indices_2 = np.array((vertices_sw, vertices_nw, vertices_ne)).T
    elif divide == "nwse":
        face_indices_1 = np.array((vertices_sw, vertices_se, vertices_nw)).T
        face_indices_2 = np.array((vertices_nw, vertices_se, vertices_ne)).T

    faces = np.concatenate((face_indices_1, face_indices_2))

    if flip:
        faces = faces[:, [0, 2, 1]]

    return vertices, faces


vertices, faces = triangulate_grid(x, y, divide="nwse")
print(vertices.tolist())
print(faces.tolist())

In [ ]:
fig, axs = plt.subplots(2, 2)

for irow, flip in enumerate([False, True]):
    for icol, divide in enumerate(["nwse", "nesw"]):
        vertices, faces = triangulate_grid(x, y, flip, divide)

        ax = axs[irow, icol]
        ax.axis("equal")
        ax.triplot(vertices[:, 0], vertices[:, 1], faces)

plt.show()
plt.close()